## TTC Subway Delay Analysis Project

In [21]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# ttc_codes = pd.read_csv('CSV Files\Code Descriptions.csv')
# ttc_codes.head()

ttc_subway_delay_24 = pd.read_csv('CSV Files/ttc-subway-delay-data-2024.csv')
# ttc_subway_delay_24.head(10)

# Updated as of Nov 24th, 2025
ttc_subway_delay_25 = pd.read_csv('CSV Files/TTC Subway Delay Data since 2025.csv')
# ttc_subway_delay_25.head(10)

ttc_codes = pd.read_json('CSV Files/Code Descriptions.json')
ttc_codes.rename(columns= {"CODE":"Code"}, inplace=True)
# ttc_codes

In [23]:
# Merged DSet
ttc_all_delays = pd.concat([ttc_subway_delay_24, ttc_subway_delay_25], ignore_index=True)
ttc_all_delays.head(10)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,_id
0,2024-01-01,02:00,Monday,SHEPPARD STATION,MUI,0,0,N,YU,5491,NaN
1,2024-01-01,02:00,Monday,DUNDAS STATION,MUIS,0,0,N,YU,0,NaN
2,2024-01-01,02:08,Monday,DUNDAS STATION,MUPAA,4,10,N,YU,6051,NaN
3,2024-01-01,02:13,Monday,KENNEDY BD STATION,PUTDN,10,16,E,BD,5284,NaN
4,2024-01-01,02:22,Monday,BLOOR STATION,MUPAA,4,10,N,YU,5986,NaN
5,2024-01-01,02:25,Monday,ST CLAIR STATION,MUPAA,3,9,N,YU,6051,NaN
6,2024-01-01,02:25,Monday,BLOOR STATION,MUIRS,0,0,S,YU,0,NaN
7,2024-01-01,02:27,Monday,WOODBINE STATION,EUDO,7,13,E,BD,5077,NaN
8,2024-01-01,02:28,Monday,FINCH STATION,MUIRS,0,0,S,YU,5561,NaN
9,2024-01-01,02:30,Monday,DAVISVILLE STATION,MUI,13,19,N,YU,6051,NaN


### Cuong's Section

Calculating average delay per type of delay at individual stations.

In [24]:

bloorDelays = ttc_all_delays.loc[ttc_all_delays['Station']  == 'BLOOR STATION']
bloorDelays.head()


,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,_id
4,2024-01-01,02:22,Monday,BLOOR STATION,MUPAA,4,10,N,YU,5986,NaN
6,2024-01-01,02:25,Monday,BLOOR STATION,MUIRS,0,0,S,YU,0,NaN
34,2024-01-01,14:15,Monday,BLOOR STATION,MUIR,12,16,N,YU,5631,NaN
40,2024-01-01,17:17,Monday,BLOOR STATION,SUDP,0,0,S,YU,0,NaN
51,2024-01-01,00:48,Monday,BLOOR STATION,SUO,0,0,S,YU,5776,NaN


In [25]:
bloorDelays = bloorDelays.groupby('Code')[['Min Delay']].agg(['mean', 'min', 'max']).sort_values(('Min Delay', 'mean'), ascending=False)
print (bloorDelays[59:61])

      Min Delay        
           mean min max
Code                   
TUCC        0.0   0   0
TUNCA       0.0   0   0


In [26]:
desc= []

for r in bloorDelays.itertuples(index=True):
    flag = True
    for l in ttc_codes.itertuples(index=False):
        if (r.Index == l.Code):
            flag = False
            desc.append(l.DESCRIPTION)
    if(flag):
        desc.append('ERR')

bloorDelays['Description'] = desc
print(bloorDelays)


      Min Delay                                                Description
           mean min max                                                   
Code                                                                      
MUPR1      48.0  48  48      PRIORITY ONE â TRAIN IN CONTACT WITH PERSON
SUBT       40.0  40  40                                        BOMB THREAT
PUTD       40.0  40  40               DEBRIS AT TRACK LEVEL - CONTROLLABLE
PUTDN      32.0   0  58         DEBRIS AT TRACK LEVEL â NON-CONTROLLABLE
PUSZC      29.0  29  29                    SIGNALS ZONE CONTROLLER FAILURE
...         ...  ..  ..                                                ...
PUMEL       0.0   0   0  ANY ESCL. OR ELEV. INCIDENT â TRANSPORTED OR...
SUSA        0.0   0   0                                   ASSAULT / SEXUAL
SUSP        0.0   0   0                                 SUSPICIOUS PACKAGE
TUCC        0.0   0   0  TRANSIT CONTROL RELATED â TOWER CONTROLLER E...
TUNCA       0.0   0   0  

In [27]:
bloorDelays.loc[bloorDelays['Description'] == 'ERR']

Min Delay         Description
          mean min max            
Code                              
PUTO  4.428571   0   7         ERR

### Megans Section

##### General Thoughts
- We might want to come up with further subcategories for what is "Maintenance" / "Patron Related (Sickness)" / "Track Issues" other issues

#### Looking at the Merged Data Further

We see that there are 664 unique "station" names. This means there are anomolies/deviations from the traditional station naming process. Once such example is: St.George Station to College Station. From this we learn they aggregate multi-station delays into the [beginning] to [ending] station.

In [28]:
# Station Masterlist, In Order

# Line 2
LINE_2 = [
    "KIPLING","ISLINGTON", "ROYAL YORK", "OLD MILL", "JANE", "RUNNYMEDE", "HIGH PARK", "KEELE", "DUFFERIN","OSSINGTON", "CHRISTIE",
    "BATHURST","SPADINA","ST GEORGE","BAY","YONGE","SHERBOURNE","CASTLE FRANK","BROADVIEW",
    "CHESTER","PAPE", "DONLANDS","GREENWOOD","COXWELL","WOODBINE","MAIN STREET","VICTORIA PARK","WARDEN","KENNEDY BD"
]
# Line 1 --> doesnt make sense, once it hits union it switched ....
LINE_1 = ["FINCH","NORTH YORK CENTRE","SHEPPARD YONGE","YORK MILLS","LAWRENCE","EGLINTON","DAVISVILLE","ST CLAIR","SUMMERHILL","ROSEDALE",
    "BLOOR YONGE","WELLESLEY","COLLEGE","DUNDAS","QUEEN","KING","UNION","ST ANDREW","OSGOODE","ST PATRICK",
    "QUEENS PARK","MUSEUM","ST GEORGE","SPADINA","DUPONT","ST CLAIR WEST","EGLINTON WEST","GLENCAIRN","LAWRENCE WEST","YORKDALE",
    "WILSON","SHEPPARD WEST","DOWNSVIEW PARK","FINCH WEST","YORK UNIVERSITY","PIONEER VILLAGE","HIGHWAY 407","VAUGHAN MC"
]
# Note, sometimes BLOOR YONGE is logged as YO
# ST A or ST An is St Andrew

In [29]:
# pd.set_option('display.max_rows', None)
print(len(ttc_all_delays['Station'].unique())) # 664 Unique Station Delay Row Names...

664


In [30]:
# Filtering for the Ranged Stations using " TO "
range_stations = ttc_all_delays[
    # Exceptions for spaces and other things
    ttc_all_delays['Station'].str.contains(r"\s+to\s+", case=False, na=False)
].copy()

# NVM keeping it in, its a hassle

# Removing the word STATION for readability
#  range_stations['Station'] = (
#     range_stations['Station']
#         .str.replace(r"\bSTAT(ION)?\b|\bSTN\b|\bSTA\b|\bST\b", "", regex = True)
#         .str.strip()
# )

# Only Yonge University Line, merging in code descriptions

# Only 217 Instances from 2024-2025
# you need to also include dashes
range_stations_YU = range_stations[range_stations["Line"] == "YU"].copy()
range_stations_YU = range_stations_YU.merge(ttc_codes, how = "left", on="Code")
range_stations_YU.head()


,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,_id_x,_id_y,DESCRIPTION
0,2024-01-10,13:30,Wednesday,DAVISVILLE TO EGLINTON,MUO,0,0,N,YU,0,NaN,51.0,MISCELLANEOUS OTHER
1,2024-01-15,23:00,Monday,EGLINTON TO YORK MILLS,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
2,2024-01-16,23:00,Tuesday,EGLINTON STATION TO YO,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER
3,2024-01-16,23:20,Tuesday,DAVISVILLE TO EGLINTON,PUTO,13,20,NaN,YU,0,NaN,NaN,NaN
4,2024-01-17,23:01,Wednesday,EGLINTON TO YORK MILLS,MUO,0,0,NaN,YU,0,NaN,51.0,MISCELLANEOUS OTHER


In [31]:
summary_YU_Delays = (
    range_stations_YU.groupby("DESCRIPTION", as_index = False)
    .agg(
        total_delay_min = ("Min Delay", "sum"),
        avg_delay_min   = ("Min Delay", "mean"),
        # max_delay_min   = ("Min Delay", "max"),
        delay_count  = ("Min Delay", "size"),
        total_delay_gap_min = ("Min Gap", "sum"),
    )
)
summary_YU_Delays.sort_values('total_delay_min')
# descrip_summary = range_stations_YU.groupby('DESCRIPTION')['Min Delay'].sum().sort_values()
# descrip_summary


,DESCRIPTION,total_delay_min,avg_delay_min,delay_count,total_delay_gap_min
1,CLOSURES - PLANNED,0,0.000000,25,0
15,NO COLLECTOR AVAILABLE,0,0.000000,1,0
10,INJURED/ILL CUSTOMER IN STATION â MEDICAL AI...,0,0.000000,1,0
9,INJURED/ILL CUSTOMER IN STATION - TRANSPORTED,0,0.000000,1,0
13,MISCELLANEOUS GENERAL SUBWAY LINE DELAYS -,0,0.000000,4,0
30,WORK REFUSAL,0,0.000000,1,0
27,TRANSIT CONTROL RELATED â TOWER CONTROLLER E...,0,0.000000,1,0
19,PLANT MAINTENANCE â OTHER,0,0.000000,1,0
4,DOOR PROBLEMS RE:FAULTY EQUIPMENT,4,4.000000,1,8
2,CONTROLS IMPROPERLY BLOWN DOWN,4,4.000000,1,0


#### Analyzing Stations at StGeorge to Bloor Yonge

In [32]:
# Total Minimum Delay is 2265 minutes
range_stations['Min Delay'].sum()

# Total Minimum Gap is 2365 minutes
range_stations['Min Gap'].sum()

np.int64(2365)

#### Basic Delays Analysis

Minimum Delays - Delay (in minutes) to subway service

Min Gap - Time length (in minutes) between trains (likely caused by the delay)

In [33]:
delays = ttc_all_delays.groupby('Station')
# delays.size().sort_values(ascending=False).head(50).sort_values()

delays_mins = ttc_all_delays.groupby('Station')['Min Delay'].sum().sort_values(ascending = False)
delays_mins

Station
EGLINTON STATION          4594
KIPLING STATION           4145
KENNEDY BD STATION        4012
BLOOR STATION             3737
FINCH STATION             3665
                          ... 
YORK UNIVERISTY TO PIO       0
YORK MILLS TO ST CLAIR       0
YORK MILLS TO SHEPPARD       0
1900 YONGE ST- MCBRIEN       0
YU-ST CLAIR STATION TO       0
Name: Min Delay, Length: 664, dtype: int64

In [34]:
delays_gap = ttc_all_delays.groupby('Station')['Min Gap'].sum().sort_values(ascending = False)
delays_gap

Station
KIPLING STATION           6639
KENNEDY BD STATION        6388
EGLINTON STATION          6210
BLOOR STATION             5749
FINCH STATION             5679
                          ... 
FINCH TO EGLINTON            0
APPROACHING FINCH STAT       0
APPROACHING DUNDAS WES       0
ALL STATIONS                 0
YORK MILLS TO EGLINTON       0
Name: Min Gap, Length: 664, dtype: int64